In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
import yfinance as yf
import time as time
from tabulate import tabulate  # Import tabulate from the tabulate library

# Defining Stock Data

Importing Symbols

In [9]:
# # Importing Symbols File

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [21]:
# Read the Excel data (NSE Stocks) into a DataFrame
df = pd.read_csv('bullish.csv', header=0)
df.head()

,Symbol
0,MANORG.NS
1,REGENCERAM.NS
2,MADHAV.NS
3,SHREYANIND.NS
4,HAVELLS.NS


In [22]:
# symbols = df[df['Signal'] == 'Bullish']['Symbol']
symbols= df['Symbol']

In [23]:
symbols.head()

0        MANORG.NS
1    REGENCERAM.NS
2        MADHAV.NS
3    SHREYANIND.NS
4       HAVELLS.NS
Name: Symbol, dtype: object

In [24]:
# @title Timeframe

interval="1d"

X=20             # Length 1
Y=50            # Length 2

start="2023-05-01"
end="2024-04-30"

# interval can be 1m,1h,1d,1w,1m
# maximum of 7day data can be fetched from yahoo in single request if interval is of 1 minute


In [25]:
# @title Capital and Brokerage

Capital = 2000  # Initial capital
brokerage_rate = 0.010  # Brokerage rate as a decimal (1%)

# MA Strategy Returns

In [26]:
# @title MA Code

# Initialize a list to store results for each symbol
results = []
initial_capital = Capital

# Initialize counters
total_symbols = len(symbols)
symbols_processed = 0


# Loop through each symbol
for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        market_cap = info.get('marketCap', 'N/A')
        market_cap /=10000000                       ## For Market Cap. in Crores
        data = yf.download(symbol, start=start, end=end, interval=interval)  # Load data for the current symbol

        # Check if data is empty for the specified date range
        if data.empty:
          print(f"Data not available for {symbol}. Skipping...")
          symbols_processed += 1
          symbols_remaining = total_symbols - symbols_processed
          continue  # Skip to the next symbol

        # Calculate the 50-day and 100-day EMA
        data['50EMA'] = data['Close'].rolling(X).mean()
        data['100EMA'] = data['Close'].rolling(Y).mean()

        # Initialize variables to track trade information
        entry_price = None
        exit_price = None
        in_trade = False
        profit = 0
        positive_trades=0
        cumulative_profit = 0
        capital_records = []

        # Implement the trading strategy
        for i in range(1, len(data)):
          if data['50EMA'][i] > data['100EMA'][i] and not in_trade:
            # Buy signal
            entry_price = data['Close'][i]
            in_trade = True
            # Calculate quantity based on available capital
            quantity = capital // entry_price

          elif data['50EMA'][i] < data['100EMA'][i] and in_trade:
            # Sell signal
            exit_price = data['Close'][i]
            in_trade = False
            trade_profit = quantity * (exit_price - entry_price)

            factor=(exit_price + entry_price)/2
            # Calculate brokerage fees and deduct them
            brokerage = quantity * factor * brokerage_rate
            trade_profit -= brokerage

            profit += trade_profit
            cumulative_profit += trade_profit
            capital += trade_profit  # Reinvest profit into capital

            capital_records.append(capital)

            #Calculating profitability
            if(trade_profit>0):
              positive_trades+=1

        # Check if the last position is still open and add it to the records
        if in_trade:
          last_price = data['Close'][-1]
          last_profit = quantity * (last_price - entry_price)
          factor=(last_price + entry_price)/2
          # Calculate brokerage fees and deduct them
          brokerage = quantity * factor * brokerage_rate
          last_profit -= brokerage
          profit += last_profit
          cumulative_profit += last_profit
          capital += last_profit  # Reinvest profit into capital

          capital_records.append(capital)

        # Calculate profitability percentage
        profitability_percentage = (positive_trades / len(capital_records)) * 100

        # Calculate maximum drawdown percentage
        data = capital_records
        drawdowns = []
        first_drawdown=(data[0]-initial_capital)/initial_capital*100
        drawdowns.append(first_drawdown)

        for i in range(1, len(data)):
          old_value = data[i - 1]
          new_value = data[i]
          drawdown = ((new_value - old_value) / old_value) * 100
          drawdowns.append(drawdown)

        max_drawdown=min(drawdowns)

        # Calculate the percentage of return on initial capital
        return_on_initial_capital = (capital - initial_capital) / initial_capital * 100

        Trades=len(capital_records)
        # Append results for the current symbol to the results list
        results.append([symbol, market_cap, return_on_initial_capital, profitability_percentage, max_drawdown,Trades])

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

    # Resetting the Capital for next Symbol
    capital=initial_capital


# Update the progress counter
    symbols_processed += 1
    symbols_remaining = total_symbols - symbols_processed
    print(f"Processed {symbols_processed}/{total_symbols} symbols. {symbols_remaining} remaining.")

# Create a DataFrame for the results
result_columns = ['Symbol','Market Cap.(Cr.)', 'Return%', 'Profitability%', 'Max. drawdown%','Trades']
result_df = pd.DataFrame(results, columns=result_columns)


[*********************100%%**********************]  1 of 1 completed


Processed 1/550 symbols. 549 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 2/550 symbols. 548 remaining.


[*********************100%%**********************]  1 of 1 completed


...

Processed 548/550 symbols. 2 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 549/550 symbols. 1 remaining.


[*********************100%%**********************]  1 of 1 completed

Processed 550/550 symbols. 0 remaining.


In [27]:
# @title Results

# Create a summary DataFrame for the percentages
summary = pd.DataFrame({
    'Average Type': 'MA',
    'Length_1': [X],  # Wrap X in a list to make it a scalar value inside a list
    'Length_2': [Y],  # Wrap Y in a list to make it a scalar value inside a list
    'Average Return(%)': [result_df['Return%'].mean()],  # Wrap the mean value in a list
    'Average Profitability(%)': [result_df['Profitability%'].mean()],  # Wrap the mean value in a list
    'Average MaxDrawdown(%)': [result_df['Max. drawdown%'].mean()]  # Wrap the mean value in a list
}, index=[0])  # Set the index to 0


# Display the percentage summary DataFrame as a formatted table
print("\nSummary:")
table = tabulate(summary, headers='keys', tablefmt='pretty', showindex=False)
print(table)


# Convert specific columns to integers (no decimal places)
columns_to_convert = ['Market Cap.(Cr.)', 'Return%', 'Profitability%', 'Max. drawdown%']

# Set the float format to display all digits
pd.options.display.float_format = '{:.2f}'.format

for col in columns_to_convert:
  result_df[col] = result_df[col].round(2)

result_df = result_df.sort_values(by='Return%', ascending=False)
result_df


Summary:
+--------------+----------+----------+--------------------+--------------------------+------------------------+
| Average Type | Length_1 | Length_2 | Average Return(%)  | Average Profitability(%) | Average MaxDrawdown(%) |
+--------------+----------+----------+--------------------+--------------------------+------------------------+
|      MA      |    20    |    50    | 30.766858149534634 |    38.03582270795384     |   4.061953278160222    |
+--------------+----------+----------+--------------------+--------------------------+------------------------+


,Symbol,Market Cap.(Cr.),Return%,Profitability%,Max. drawdown%,Trades
481,TRIL.BO,9266.66,619.29,0.00,619.29,1
279,HIMFIBP.BO,378.55,557.19,100.00,557.19,1
491,JAIBALAJI.BO,17417.45,539.33,100.00,539.33,1
395,DHRUVCA.BO,140.06,521.92,0.00,521.92,1
397,TRISHAKT.BO,19.29,488.84,0.00,-8.65,2
...,...,...,...,...,...,...
411,GOGIACAP.BO,59.99,-48.88,25.00,-28.75,4
200,SKYLMILAR.BO,79.00,-53.87,0.00,-32.00,4
112,SERVOTECH.NS,158.29,-55.69,0.00,-47.09,4
14,CUPID.NS,2865.71,-71.47,0.00,-69.81,2


## **Exporting Results**

In [28]:
# Export the DataFrame to a CSV file
csv_file_path = "Results_MA_returns.csv"  # Change the file path as needed
result_df.to_csv(csv_file_path, index=False)

# EMA Strategy Returns

In [29]:
# @title EMA Code

# Initialize a list to store results for each symbol
results = []
initial_capital = Capital

# Initialize counters
total_symbols = len(symbols)
symbols_processed = 0


# Loop through each symbol
for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        market_cap = info.get('marketCap', 'N/A')
        market_cap /=10000000                       ## For Market Cap. in Crores
        data = yf.download(symbol, start=start, end=end, interval=interval)  # Load data for the current symbol

        # Check if data is empty for the specified date range
        if data.empty:
          print(f"Data not available for {symbol}. Skipping...")
          symbols_processed += 1
          symbols_remaining = total_symbols - symbols_processed
          continue  # Skip to the next symbol

        # Calculate the 50-day and 100-day EMA
        data['50EMA'] = data['Close'].ewm(span=X, adjust=False).mean()
        data['100EMA'] = data['Close'].ewm(span=Y, adjust=False).mean()

        # Initialize variables to track trade information
        entry_price = None
        exit_price = None
        in_trade = False
        profit = 0
        positive_trades=0
        cumulative_profit = 0
        capital_records = []

        # Implement the trading strategy
        for i in range(1, len(data)):
          if data['50EMA'][i] > data['100EMA'][i] and not in_trade:
            # Buy signal
            entry_price = data['Close'][i]
            in_trade = True
            # Calculate quantity based on available capital
            quantity = capital // entry_price

          elif data['50EMA'][i] < data['100EMA'][i] and in_trade:
            # Sell signal
            exit_price = data['Close'][i]
            in_trade = False
            trade_profit = quantity * (exit_price - entry_price)

            factor=(exit_price + entry_price)/2
            # Calculate brokerage fees and deduct them
            brokerage = quantity * factor * brokerage_rate
            trade_profit -= brokerage

            profit += trade_profit
            cumulative_profit += trade_profit
            capital += trade_profit  # Reinvest profit into capital

            capital_records.append(capital)

            #Calculating profitability
            if(trade_profit>0):
              positive_trades+=1

        # Check if the last position is still open and add it to the records
        if in_trade:
          last_price = data['Close'][-1]
          last_profit = quantity * (last_price - entry_price)
          factor=(last_price + entry_price)/2
          # Calculate brokerage fees and deduct them
          brokerage = quantity * factor * brokerage_rate
          last_profit -= brokerage
          profit += last_profit
          cumulative_profit += last_profit
          capital += last_profit  # Reinvest profit into capital

          capital_records.append(capital)

        # Calculate profitability percentage
        profitability_percentage = (positive_trades / len(capital_records)) * 100

        # Calculate maximum drawdown percentage
        data = capital_records
        drawdowns = []
        first_drawdown=(data[0]-initial_capital)/initial_capital*100
        drawdowns.append(first_drawdown)

        for i in range(1, len(data)):
          old_value = data[i - 1]
          new_value = data[i]
          drawdown = ((new_value - old_value) / old_value) * 100
          drawdowns.append(drawdown)

        max_drawdown=min(drawdowns)

        # Calculate the percentage of return on initial capital
        return_on_initial_capital = (capital - initial_capital) / initial_capital * 100

        Trades=len(capital_records)
        # Append results for the current symbol to the results list
        results.append([symbol, market_cap, return_on_initial_capital, profitability_percentage, max_drawdown,Trades])

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

    # Resetting the Capital for next Symbol
    capital=initial_capital


# Update the progress counter
    symbols_processed += 1
    symbols_remaining = total_symbols - symbols_processed
    print(f"Processed {symbols_processed}/{total_symbols} symbols. {symbols_remaining} remaining.")

# Create a DataFrame for the results
result_columns = ['Symbol','Market Cap.(Cr.)', 'Return%', 'Profitability%', 'Max. drawdown%','Trades']
result_df = pd.DataFrame(results, columns=result_columns)


[*********************100%%**********************]  1 of 1 completed


Processed 1/550 symbols. 549 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 2/550 symbols. 548 remaining.


[*********************100%%**********************]  1 of 1 completed


...

Processed 548/550 symbols. 2 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 549/550 symbols. 1 remaining.


[*********************100%%**********************]  1 of 1 completed

Processed 550/550 symbols. 0 remaining.


In [30]:
# @title Results

# Create a summary DataFrame for the percentages
summary = pd.DataFrame({
    'Average Type': 'EMA',
    'Length_1': [X],  # Wrap X in a list to make it a scalar value inside a list
    'Length_2': [Y],  # Wrap Y in a list to make it a scalar value inside a list
    'Average Return(%)': [result_df['Return%'].mean()],  # Wrap the mean value in a list
    'Average Profitability(%)': [result_df['Profitability%'].mean()],  # Wrap the mean value in a list
    'Average MaxDrawdown(%)': [result_df['Max. drawdown%'].mean()]  # Wrap the mean value in a list
}, index=[0])  # Set the index to 0


# Display the percentage summary DataFrame as a formatted table
print("\nSummary:")
table = tabulate(summary, headers='keys', tablefmt='pretty', showindex=False)
print(table)


# Convert specific columns to integers (no decimal places)
columns_to_convert = ['Market Cap.(Cr.)', 'Return%', 'Profitability%', 'Max. drawdown%']

# Set the float format to display all digits
pd.options.display.float_format = '{:.2f}'.format

for col in columns_to_convert:
  result_df[col] = result_df[col].round(2)

result_df = result_df.sort_values(by='Return%', ascending=False)
result_df


Summary:
+--------------+----------+----------+-------------------+--------------------------+------------------------+
| Average Type | Length_1 | Length_2 | Average Return(%) | Average Profitability(%) | Average MaxDrawdown(%) |
+--------------+----------+----------+-------------------+--------------------------+------------------------+
|     EMA      |    20    |    50    | 47.28237150368233 |    28.30617831074039     |   8.013818639299139    |
+--------------+----------+----------+-------------------+--------------------------+------------------------+


,Symbol,Market Cap.(Cr.),Return%,Profitability%,Max. drawdown%,Trades
489,JAIBALAJI.BO,17296.73,1143.03,50.00,-1.51,2
479,TRIL.BO,9280.20,751.46,0.00,-9.19,2
292,RSSOFTWARE.BO,682.81,732.31,0.00,732.31,1
314,RAJOOENG.BO,1525.96,594.31,50.00,20.40,2
393,DHRUVCA.BO,140.06,586.11,0.00,586.11,1
...,...,...,...,...,...,...
285,PATSPINLTD.BO,43.13,-49.02,0.00,-20.18,4
409,GOGIACAP.BO,59.99,-51.57,0.00,-35.06,3
287,CALCOM.BO,150.67,-52.06,0.00,-16.87,9
340,SHUKRAPHAR.BO,14.91,-55.17,25.00,-37.48,4


## **Exporting Results**

In [31]:
# Export the DataFrame to a CSV file
csv_file_path = "Results_EMA_returns.csv"  # Change the file path as needed
result_df.to_csv(csv_file_path, index=False)